In [114]:
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle

#Seaborn and matplotlib
import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

#Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

import warnings
warnings.filterwarnings("ignore")

In [115]:
INPUT_DIR = '../Data'
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
selling_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

What to predict

## Train - validation - test

In [116]:
1913-28

1885

In [117]:
1884-90

1794

In [118]:
d_cols = [c for c in sales_train_val.columns if 'd_' in c]

train_dataset = sales_train_val[d_cols[1794:1885]]
val_dataset = sales_train_val[d_cols[1885:]]

In [119]:
x_train=np.arange(1794,1885)
x_valid=np.arange(1885,1913+1)

In [120]:
len(train_dataset.loc[0].values)

91

In [121]:
fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(90), mode='lines', y=train_dataset.loc[8412].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(90, 118), y=val_dataset.loc[8412].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[8413].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[8413].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[8414].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[8414].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Train (blue) vs. Validation (orange) sales")
fig.show()

## Moving average <a id="3.3"></a>

The **moving average** method is more complex than the naive approach. It calculates the mean sales over the previous 30 days and forecasts that as the next day's sales. This method takes the previous 30 timesteps into consideration, and is therefore less prone to short term fluctuations than the naive approach. The model can be summarized as follows:

<img src="https://i.imgur.com/5uJvt7H.png" width="200px">

In the above equation, *y<sub>t+1</sub>* is tomorrow's sales. On the right hand side, all the sales for the previous 30 days are added up and divided by 30 to find the average. This forms the model's prediction, *y<sub>t+1</sub>*. Now let us see how this new model performs on our miniature dataset. The training data is in <font color="blue">blue</font>, validation data in <font color="darkorange">orange</font>, and predictions in <font color="green">green</font>.

In [130]:
predictions = []
for i in range(len(val_dataset.columns)):
    if i == 0:
        predictions.append(np.mean(train_dataset[train_dataset.columns[-28:]].values, axis=1))
    if i < 31 and i > 0:
        predictions.append(0.5 * (np.mean(train_dataset[train_dataset.columns[-28+i:]].values, axis=1) + \
                                  np.mean(predictions[:i], axis=0)))
    if i > 31:
        predictions.append(np.mean([predictions[:i]], axis=1))
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_avg = np.linalg.norm(predictions[:3] - val_dataset.values[8412:8415])/len(predictions[0])

In [131]:
error_avg

21.636155436146826

In [129]:
val_dataset.values[8412:8415]

array([[ 61,  62,  71, 130, 156, 133, 114,  80,  75, 113,  81, 107, 160,
        124, 107,  84,  88,  90, 129, 160, 204,  86, 100,  88,  77, 141,
        139, 130],
       [  2,   2,   1,   2,   2,   0,   2,   4,   1,   0,   0,   1,   3,
          0,   3,   0,   1,   1,   1,   0,   2,   0,   0,   2,   0,   3,
          2,   4],
       [  1,   6,   1,   2,   1,   2,   1,   2,   3,   0,   0,   3,   1,
          1,   2,   1,   0,   0,   7,   3,   2,   0,   1,   1,   0,   0,
          4,   1]])

In [125]:
predictions

array([[212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674],
       [  1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329],
       [  1.53333335,   1.5333

In [123]:
pred_1 = predictions[8412]
pred_2 = predictions[8413]
pred_3 = predictions[8414]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[8412].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[8412].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[8413].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[8413].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[8414].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[8414].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Moving average")
fig.show()

## Exponential smoothing <a id="3.5"></a>

The **exponential smoothing** method uses a different type of "smoothing" which differs from average smoothing. The previous time steps are exponentially weighted and added up to generate the forecast. The weights decay as we move further backwards in time. The model can be summarized as follows:

<img src="https://i.imgur.com/IqqjOFc.png" width="520px">
<img src="https://i.imgur.com/GiyHyZf.png" width="255px">

In the above equations, $\alpha$ is the smoothing parameter. The forecast *y<sub>t+1</sub>* is a weighted average of all the observations in the series *y<sub>1</sub>, … ,y<sub>t</sub>*. The rate at which the weights decay is controlled by the parameter *α*. This method gives different weightage to different time steps, instead of giving the same weightage to all time steps (like the moving average method). This ensures that recent sales data is given more importance than old sales data while making the forecast. Now let us see how this new smoothing method performs on our miniature dataset. The training data is in <font color="blue">blue</font>, validation data in <font color="darkorange">orange</font>, and predictions in <font color="green">green</font>.

In [127]:
val_dataset.values[8412:8415]

array([[ 61,  62,  71, 130, 156, 133, 114,  80,  75, 113,  81, 107, 160,
        124, 107,  84,  88,  90, 129, 160, 204,  86, 100,  88,  77, 141,
        139, 130],
       [  2,   2,   1,   2,   2,   0,   2,   4,   1,   0,   0,   1,   3,
          0,   3,   0,   1,   1,   1,   0,   2,   0,   0,   2,   0,   3,
          2,   4],
       [  1,   6,   1,   2,   1,   2,   1,   2,   3,   0,   0,   3,   1,
          1,   2,   1,   0,   0,   7,   3,   2,   0,   1,   1,   0,   0,
          4,   1]])

In [126]:
predictions

array([[212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674, 212.10277674, 212.10277674,
        212.10277674, 212.10277674],
       [  1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329,   1.93333329,   1.93333329,
          1.93333329,   1.93333329],
       [  1.53333335,   1.5333

In [124]:
predictions = []
for row in train_dataset[train_dataset.columns[-30:]].values[8412:8415]:
    row=row.astype(np.double)
    fit = ExponentialSmoothing(row, seasonal_periods=3).fit()
    predictions.append(fit.forecast(30))

predictions = np.array(predictions).reshape((-1, 30))

error_exponential = np.linalg.norm(predictions[:3] - val_dataset.values[8412:8415])/len(predictions[0])

ValueError: operands could not be broadcast together with shapes (3,30) (3,28) 

In [ ]:
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[0].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[0].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Exponential smoothing")
fig.show()

## ARIMA <a id="3.6"></a>

**ARIMA** stands for **A**uto **R**egressive **I**ntegrated **M**oving **A**verage. While exponential smoothing models were based on a description of trend and seasonality in data, ARIMA models aim to describe the correlations in the time series. The video below explains ARIMA very well:

In [ ]:
predictions = []
for row in train_dataset[train_dataset.columns[-30:]].values[:3]:
    fit = sm.tsa.statespace.SARIMAX(row, seasonal_order=(0, 1, 1, 7)).fit()
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
error_arima = np.linalg.norm(predictions[:3] - val_dataset.values[:3])/len(predictions[0])

In [ ]:
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[0].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[0].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="ARIMA")
fig.show()

In [ ]:
error = [error_avg, error_exponential, error_arima]
names = ["Moving average",  "Exponential smoothing", "ARIMA"]
df = pd.DataFrame(np.transpose([error, names]))
df.columns = ["RMSE Loss", "Model"]
px.bar(df, y="RMSE Loss", x="Model", color="Model", title="RMSE Loss vs. Model")